In [83]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool, Dropdown, Select
from bokeh.layouts import widgetbox, row, column
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
import numpy as np
import panel as pn
import holoviews as hv
import holoviews.plotting.bokeh
import pandas as pd
import geopandas as gpd
import json

In [113]:
# Read the shape file for districts (Kreise) and states (Bundesländer)
gd_krs = gpd.read_file(r'.\mapdata\vg2500_krs.shp')
gd_bld = gpd.read_file(r'.\mapdata\vg2500_bld.shp')

In [122]:
# Read the data
# Here we create random data
df1 = pd.DataFrame(np.random.randint(50,100,size=(len(gd_krs), 1)), columns=list('N'))
df1['TYPE'] = 'type1'
df1['RS'] = gd_krs['RS']

df2 = pd.DataFrame(np.random.randint(50,100,size=(len(gd_krs), 1)), columns=list('N'))
df2['TYPE'] = 'type2'
df2['RS'] = gd_krs['RS']

df = pd.concat([df1, df2])
df.head(5)

,N,TYPE,RS
0,74,type1,01001
1,76,type1,01002
2,57,type1,01003
3,96,type1,01004
4,65,type1,01051


In [124]:
#Define function that returns json_data, lowest value, highest value and 
def json_data(var1):
    # Filter for var1
    df2 = df[df['TYPE'] == var1]
    # Define lowest and highest value
    low_value = df2['N'].min()
    high_value = df2['N'].max()
    
    tick_labels = {}
    for l in range(int(low_value), int(high_value), int((high_value-low_value)/3)):
        tick_labels.update({str(l):str(l)})
    
    # Merge data and geo data and transform it to JSON
    merged = gd_krs.merge(df2, on = 'RS', how = 'left')
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    
    return json_data, low_value, high_value, tick_labels

# Execute the function
df_uniq = df['TYPE'].unique() 
geojs, low_val, high_val, tick_labels = json_data(df_uniq[0])

# Bundesländer border geo data
gd_bld_json = json.loads(gd_bld.to_json())
json_data_bl = json.dumps(gd_bld_json)

# Input GeoJSON source
geosource = GeoJSONDataSource(geojson = geojs)
geosource_bl = GeoJSONDataSource(geojson = json_data_bl)

# Define the color palette
palette = brewer['YlGnBu'][8]

# Instantiate the ColorMapper
color_mapper = LinearColorMapper(palette = palette, low = low_val, high = high_val, nan_color = '#d9d9d9')

# Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20, border_line_color=None,location = (0,0),
                     orientation = 'horizontal', major_label_overrides = tick_labels)

# Create an object
p = figure(title = 'Plot title', plot_height = 850 , plot_width = 700, toolbar_location = None)#, tools = [hover])
p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add renderer to figure
kr = p.patches('xs','ys', source = geosource,fill_color = {'field' :'N', 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)
#p.patches('xs','ys', source = geosource_bl, line_color = 'black', line_width = 1, fill_alpha = 0)
hover = HoverTool(renderers=[kr],tooltips = [ ('Kreis','@RS'),('N', '@N{0}')])
p.add_tools(hover)
p.add_layout(color_bar, 'below')

# Define the callback function: update_plot
def update_plot(attr, old, new):
    value1 = dropdown_1.value
    new_data_j, new_data_l, new_data_h, new_data_t = json_data(value1)
    geosource.geojson = new_data_j
    color_mapper.low = new_data_l
    color_mapper.high = new_data_h
    color_bar.major_label_overrides = new_data_t

# Create the select
menu_df = []
for i in (df['TYPE'].unique()):
    menu_df.append((i))
menu_df.sort()
dropdown_1 = Select(title='TYPE', value=menu_df[0], options=menu_df)
dropdown_1.on_change('value', update_plot)

# Create the layout
layout = column(dropdown_1,p)
curdoc().add_root(layout)

# Display plot inline in Jupyter notebook
# output_notebook()
# show(layout)

# Display plot in browser
server = pn.serve(layout, start=True, show=True)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

